## Importing modules

In [1]:
import h5py
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.models import resnet50
import numpy as np
from functools import partial

import skimage
from skimage.color import rgb2hed
import pywt
from PIL import Image
import cv2
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchstain

from histomicstk.preprocessing.color_normalization.deconvolution_based_normalization import deconvolution_based_normalization


In [2]:
class H5Dataset(Dataset):
    def __init__(self, image_file, label_file, transform=None):
        self.transform = transform
        
        # Load data from the H5 file
        with h5py.File(image_file, 'r') as f:
            self.images = f['x'][:]
        with h5py.File(label_file, 'r') as f:
            self.labels = f['y'][:].reshape(-1,)
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [3]:
class RGB2HED(torch.nn.Module):
    def __init__(self, mode=None):
        super(RGB2HED, self).__init__()
        self.mode = mode
    def forward(self, img):
        img = img.astype(np.float32) / 255.
        hed_img = rgb2hed(img) * 255.
        hed_img = np.tile(hed_img[:, :, -2:-1], reps=(1,1,3))
        return hed_img
    
        
class WaveletTransform(nn.Module):
    def __init__(self, wavelet='haar', threshold=20):
        super(WaveletTransform, self).__init__()
        self.wavelet = wavelet
        self.threshold = threshold
        
    def forward(self, img):
        grayscale_image = np.dot(img.astype(np.uint8), [0.299, 0.587, 0.114])
        
        # Step 2: Perform 2D wavelet decomposition
        coeffs = pywt.wavedec2(grayscale_image, wavelet=self.wavelet, level=2)
        cA, details = coeffs[0], coeffs[1:]
        
        # Step 3: Apply thresholding to detail coefficients
        def threshold_coeffs(coeffs, threshold):
            return [pywt.threshold(c, threshold, mode='soft') for c in coeffs] 
        
        
        details_thresh = [threshold_coeffs(detail, self.threshold) for detail in details]
        coeffs_thresh = [cA] + details_thresh
        
        # Step 4: Reconstruct the image
        compressed_image = pywt.waverec2(coeffs_thresh, wavelet=self.wavelet)
        compressed_image = np.clip(compressed_image, 0, 255).astype(np.uint8)
        compressed_image = np.tile(np.expand_dims(compressed_image, -1), (1,1,3))
        
        return compressed_image
    

class CLAHE(nn.Module):
    def __init__(self, mode=None):
        super(CLAHE, self).__init__()
        self.mode = mode
    def forward(self, image):
        # Convert to LAB color space
        lab_image = cv2.cvtColor(image.astype(np.uint8), cv2.COLOR_RGB2LAB)
        l_channel, a, b = cv2.split(lab_image)

        # Apply CLAHE to the L channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l_channel = clahe.apply(l_channel)

        # Merge and convert back to RGB
        lab_image = cv2.merge((l_channel, a, b))
        return cv2.cvtColor(lab_image, cv2.COLOR_LAB2RGB)


class Opening(nn.Module):
    def __init__(self):
        super(Opening, self).__init__()
        
    def forward(self, image):
        return skimage.morphology.opening(image)
    
    
class Macenko(nn.Module):
    def __init__(self, reference_image, target_W=None, alpha=1, beta=0.01):
        super(Macenko, self).__init__()
        self.target_W = target_W
        self.alpha = alpha
        self.beta = beta
        self.reference_image = reference_image.astype(np.uint8)
    
    def forward(self, image):
        """
        Apply Macenko normalization to a single image with error handling.
        
        Parameters:
            image (np.ndarray): The image to normalize, shape (H, W, C) in RGB format.
            reference_image (np.ndarray): The reference image for normalization, shape (H, W, C) in RGB format.
        
        Returns:
            np.ndarray: The normalized image, shape (C, H, W) in normalized format.
            None: If normalization fails for any reason.
        """
        try:
            # # Set up the transformation
            # T = transforms.Compose([
            #     transforms.ToTensor(),
            # ])

            # Initialize the MacenkoNormalizer
            normalizer = torchstain.normalizers.MacenkoNormalizer(backend='torch')

            # Fit the normalizer with the reference image
            normalizer.fit(self.reference_image)

            # Transform the image and apply normalization
            t_to_transform = image
            norm_img, _, _ = normalizer.normalize(I=t_to_transform, stains=True)

            # Return the normalized image
            return norm_img

        except torch.linalg.LinAlgError as e:
            # print(f"LinAlgError during normalization: {e}")
            pass
        except Exception as e:
            # print(f"Unexpected error during normalization: {e}")
            pass

        # Return None if normalization fails
        return image

    
class ReinhardNormalization(nn.Module):
    def __init__(self, reference_image):
        super(ReinhardNormalization, self).__init__()
        self.reference_image = reference_image.astype(np.uint8)
        
    def forward(self, image):
        """
        Apply Reinhard normalization to a single image with error handling.

        Parameters:
            image (np.ndarray): The image to normalize, shape (H, W, C) in RGB format.
            reference_image (np.ndarray): The reference image for normalization, shape (H, W, C) in RGB format.

        Returns:
            np.ndarray: The normalized image, shape (H, W, C) in normalized format.
            None: If normalization fails for any reason.
        """
        try:
            # Initialize the ReinhardNormalizer
            normalizer = torchstain.normalizers.ReinhardNormalizer()

            # Fit the normalizer with the reference image
            normalizer.fit(self.reference_image)

            # Normalize the image
            normalized_image = normalizer.normalize(image)

            # Return the normalized image
            return normalized_image

        except Exception as e:
            # print(f"Unexpected error during Reinhard normalization: {e}")
            pass

        # Return None if normalization fails
        return image


train_data = H5Dataset(image_file='../../../pcam/training_split.h5', 
                        label_file='../../../Labels/Labels/camelyonpatch_level_2_split_train_y.h5')
reference_image = train_data.images[176298]

train_transform = transforms.Compose(
    [
        # RGB2HED(),
        WaveletTransform(),
        # CLAHE(),
        # Opening(),
        transforms.ToPILImage(),
        # transforms.ColorJitter(brightness=.5, saturation=.25,
        #                     hue=.1, contrast=.5),
        transforms.RandomAffine(10, (0.05, 0.05), fill=255),
        transforms.RandomHorizontalFlip(.5),
        transforms.RandomVerticalFlip(.5),
        transforms.ToTensor(),
        # Macenko(reference_image=reference_image),
        # ReinhardNormalization(reference_image=reference_image),
        transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                            [0.27210504, 0.31001145, 0.2918652]),
        
])

val_transform = transforms.Compose(
    [
        WaveletTransform(),
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.6716241, 0.48636872, 0.60884315],
                            [0.27210504, 0.31001145, 0.2918652])
        
    ]
)

In [4]:
# Load datasets
train_dataset = H5Dataset(image_file='../../../pcam/training_split.h5', 
                          label_file='../../../Labels/Labels/camelyonpatch_level_2_split_train_y.h5', 
                          transform=train_transform)
val_dataset = H5Dataset(image_file='../../../pcam/validation_split.h5', 
                        label_file='../../../Labels/Labels/camelyonpatch_level_2_split_valid_y.h5',
                        transform=val_transform)

test_dataset = H5Dataset(image_file='../../../pcam/test_split.h5', 
                        label_file='../../../Labels/Labels/camelyonpatch_level_2_split_test_y.h5',
                        transform=val_transform)

# Create dataloaders
bs = 128
train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=bs, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, shuffle=False, num_workers=4)

In [5]:
class ResNetModel(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNetModel, self).__init__()
        self.resnet = resnet50(pretrained=False)
        # Replace the final fully connected layer
        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.resnet(x)

In [6]:
# Initialize model, loss function, and optimizer
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = ResNetModel(num_classes=2).to(device)  # Binary classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.1)

/home/vaibhav/miniconda3/envs/debo/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vaibhav/miniconda3/envs/debo/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
# Training and validation loops
def train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            # Metrics
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
        
        # Validation phase
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Metrics
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
                
        # Test phase
        model.eval()
        test_loss, test_correct, test_total = 0, 0, 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                
                # Forward pass
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                # Metrics
                test_loss += loss.item()
                _, predicted = outputs.max(1)
                test_total += labels.size(0)
                test_correct += predicted.eq(labels).sum().item()
        
        # Print epoch results
        print(f"Epoch {epoch+1}/{epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}, Train Acc: {100 * train_correct/train_total:.2f}%")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {100 * val_correct/val_total:.2f}%")
        print(f"Test Loss: {test_loss/len(test_loader):.4f}, Test Acc: {100 * test_correct/test_total:.2f}%\n\n")

# Train and validate the model
train_and_validate(model, train_loader, val_loader, criterion, optimizer, epochs=20)

Epoch 1/20
Train Loss: 0.4239, Train Acc: 81.03%
Val Loss: 0.4726, Val Acc: 79.65%
Test Loss: 0.5289, Test Acc: 76.76%


Epoch 2/20
Train Loss: 0.3332, Train Acc: 85.76%
Val Loss: 0.4786, Val Acc: 77.86%
Test Loss: 0.5812, Test Acc: 73.40%


Epoch 3/20
Train Loss: 0.2832, Train Acc: 88.31%
Val Loss: 0.5288, Val Acc: 76.26%
Test Loss: 0.5930, Test Acc: 72.57%


Epoch 4/20
Train Loss: 0.2611, Train Acc: 89.34%
Val Loss: 0.3663, Val Acc: 86.13%
Test Loss: 0.4081, Test Acc: 83.56%


Epoch 5/20
Train Loss: 0.2457, Train Acc: 90.12%
Val Loss: 0.4506, Val Acc: 82.62%
Test Loss: 0.4823, Test Acc: 79.45%


Epoch 6/20
Train Loss: 0.2333, Train Acc: 90.68%
Val Loss: 0.4899, Val Acc: 82.52%
Test Loss: 0.4594, Test Acc: 80.94%


Epoch 7/20
Train Loss: 0.2247, Train Acc: 91.03%
Val Loss: 0.3398, Val Acc: 86.03%
Test Loss: 0.3662, Test Acc: 84.70%


Epoch 8/20
Train Loss: 0.2178, Train Acc: 91.36%
Val Loss: 0.3336, Val Acc: 86.00%
Test Loss: 0.3178, Test Acc: 86.53%


Epoch 9/20
Train Loss: 0.2128, T

In [8]:
# lr = 1e-3, wd = 0.1

# Epoch 1/20
# Train Loss: 0.4937, Train Acc: 77.01%
# Val Loss: 0.4545, Val Acc: 77.88%
# Test Loss: 0.4672, Test Acc: 77.66%


# Epoch 2/20
# Train Loss: 0.4302, Train Acc: 80.58%
# Val Loss: 0.5042, Val Acc: 77.38%
# Test Loss: 0.5756, Test Acc: 74.28%


# Epoch 3/20
# Train Loss: 0.3818, Train Acc: 83.16%
# Val Loss: 0.7236, Val Acc: 66.70%
# Test Loss: 0.7590, Test Acc: 64.38%


# Epoch 4/20
# Train Loss: 0.3425, Train Acc: 85.27%
# Val Loss: 0.3559, Val Acc: 84.76%
# Test Loss: 0.3465, Test Acc: 84.75%


# Epoch 5/20
# Train Loss: 0.3206, Train Acc: 86.45%
# Val Loss: 0.5312, Val Acc: 76.56%
# Test Loss: 0.5298, Test Acc: 74.98%


# Epoch 6/20
# Train Loss: 0.3046, Train Acc: 87.21%
# Val Loss: 0.3934, Val Acc: 83.93%
# Test Loss: 0.3418, Test Acc: 84.89%


# Epoch 7/20
# Train Loss: 0.2938, Train Acc: 87.70%
# Val Loss: 0.4143, Val Acc: 83.01%
# Test Loss: 0.4319, Test Acc: 80.88%


# Epoch 8/20
# Train Loss: 0.2844, Train Acc: 88.23%
# Val Loss: 0.5233, Val Acc: 81.82%
# Test Loss: 0.4156, Test Acc: 82.09%


# Epoch 9/20
# Train Loss: 0.2763, Train Acc: 88.61%
# Val Loss: 0.3505, Val Acc: 85.51%
# Test Loss: 0.3178, Test Acc: 86.87%


# Epoch 10/20
# Train Loss: 0.2735, Train Acc: 88.74%
# Val Loss: 0.3641, Val Acc: 84.77%
# Test Loss: 0.3637, Test Acc: 84.01%


# Epoch 11/20
# Train Loss: 0.2689, Train Acc: 88.95%
# Val Loss: 0.3803, Val Acc: 85.42%
# Test Loss: 0.3355, Test Acc: 85.45%


# Epoch 12/20
# Train Loss: 0.2664, Train Acc: 89.07%
# Val Loss: 0.3511, Val Acc: 84.93%
# Test Loss: 0.3024, Test Acc: 87.12%


# Epoch 13/20
# Train Loss: 0.2625, Train Acc: 89.28%
# Val Loss: 0.4591, Val Acc: 81.83%
# Test Loss: 0.4610, Test Acc: 81.09%


# Epoch 14/20
# Train Loss: 0.2637, Train Acc: 89.19%
# Val Loss: 0.4563, Val Acc: 82.73%
# Test Loss: 0.5160, Test Acc: 78.68%